In [9]:
import os
import keras
import pickle

In [4]:
FN = 'train'
FN0 = 'vocabulary-embedding'

In [5]:
maxlend=25 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

In [6]:
activation_rnn_size = 40 if maxlend else 0

In [7]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [19]:
nb_train_samples = 600
nb_val_samples = 400

# Read word embedding

In [10]:
with open('./data/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [11]:
with open('data/%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [12]:
nb_unknown_words = 10

In [13]:
print 'number of examples',len(X),len(Y)
print 'dimension of embedding space for words',embedding_size
print 'vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words
print 'total number of different words',len(idx2word), len(word2idx)
print 'number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx)
print 'number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx)

number of examples 1000 1000
dimension of embedding space for words 100
vocabulary size 1000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 57347 57347
number of words outside vocabulary which we can substitue using glove similarity 9359
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 46988


In [14]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [15]:
oov0 = vocab_size-nb_unknown_words

In [16]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [20]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

(600, 600, 400, 400)

In [21]:
del X
del Y

In [22]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [23]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [24]:
def prt(label, x):
    print label+':',
    for w in x:
        print idx2word[w],
    print

In [25]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H: cheers^ & jeers^
D: jeers:^ in a recent gazette^ article,^ it was announced that texarkana^ college is seeking^ a raise^ in taxes.^ the only apparent^ reason,^ that i could discern,...^ published:^ 09/12/2^


In [26]:
i = 334
prt('H',Y_test[i])
prt('D',X_test[i])

H: worcester^ breakfast^ club for veterans^ gives^ hunger^ its marching^ orders^
D: veterans^ saluted^ worcester's^ first ever breakfast^ club for ex-soldiers^ which won over hearts,^ minds^ and bellies.^ the worcester^ breakfast^ club for hm^ forces^ veterans^ met^ at the postal^ order in foregate^ street at 10am^ on saturday.^ the club is designed to allow veterans^ a place to meet,^ socialise,^ eat^ and drink,^ giving^ hunger^ and loneliness^ their marching^ orders.^ father-of-two^ dave^ carney,^ aged^ 43,^ of merrimans^ hill,^ worcester,^ set up the club after being inspired^ by other similar clubs^ across the country.^ he said: "as^ you can see from the picture,^ we had a good response.^ five out of the 10 that attended^ said they saw the article in the newspaper^ and turned up.^ "we even had an old chap^ travel from droitwich^ and he was late on parade^ by three hours.^ "it's^ generated^ a lot of interest and i estimate^ (from^ other veterans^ who saw the article)^ that next mont

# Model

In [29]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.layers import merge

In [30]:
random.seed(seed)
np.random.seed(seed)

In [31]:
regularizer = l2(weight_decay) if weight_decay else None